
# 🌼 RAG기법의 이해와 적용 - 3차시(24.12.02)

---

In [2]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory()

C:\Users\RMARKET\AppData\Local\Temp\ipykernel_17708\3262330552.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()


In [7]:
memory.save_context(
    inputs = {
        'human' : '안녕하세요 휴대폰을 구매하러 왔습니다'
    },
    outputs = {
        'ai' : '안녕하세요. 새 휴대폰을 구매하신다니 기쁩니다!'
    }
)
# 선입선출로 메모리를 읽는다

In [8]:
memory.load_memory_variables({})

{'history': 'Human: 안녕하세요 휴대폰을 구매하러 왔습니다\nAI: 안녕하세요. 새 휴대폰을 구매하신다니 기쁩니다!\nHuman: 안녕하세요 휴대폰을 구매하러 왔습니다\nAI: 안녕하세요. 새 휴대폰을 구매하신다니 기쁩니다!'}

In [12]:
from dotenv import load_dotenv

load_dotenv()

True

In [13]:
from langchain_teddynote import logging
logging.langsmith("CLASS")

LangSmith 추적을 시작합니다.
[프로젝트명]
CLASS


In [23]:
from operator import itemgetter
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI

In [24]:
model = ChatOpenAI(model = 'gpt-4o')

In [25]:
prompt = ChatPromptTemplate.from_messages(
    [
        ('system', '너는 친절한 챗봇이야'),
        MessagesPlaceholder(variable_name = 'chat_history'),
        ('human', "{input}")
    ]
)

In [26]:
memory = ConversationBufferMemory(return_messages=True, memory_key='chat_history')

In [27]:
memory.load_memory_variables({})

{'chat_history': []}

In [32]:
runnable = RunnablePassthrough.assign(
    chat_history = RunnableLambda(memory.load_memory_variables) | itemgetter('chat_history')
)
# 메모리에서 데이터를 로드 + chat_history만 get하기!!!
# 전체 입력데이터에서 chat_history값만 가져오기 -> 대화이력

In [33]:
chain = runnable | prompt | model

In [34]:
respone = chain.invoke({'input' : '만나서 반갑습니다. 제 이름은 길동입니다'})
print(respone.content)

안녕하세요, 길동님! 만나서 반갑습니다. 어떻게 도와드릴까요?


In [ ]:
memory.load_memory_variables({}) # 아직 비어있기 때문에 수동으로 넣어줘야 함

{'chat_history': []}

In [36]:
memory.save_context({'human' : '만나서 반값습니다. 제 이름은 길동입니다'}, {'ai' : respone.content})

In [37]:
respone = chain.invoke({'input' : '제 이름이 무엇이었는지 기억하시냐요?'})
print(respone.content)

네, 길동님이라고 하셨죠. 맞나요?


In [38]:
memory.save_context({'human' : '제 이름이 무엇이었는지 기억하시나요?'}, {'ai' : respone.content})

In [40]:
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='만나서 반값습니다. 제 이름은 길동입니다', additional_kwargs={}, response_metadata={}),
  AIMessage(content='안녕하세요, 길동님! 만나서 반갑습니다. 어떻게 도와드릴까요?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='제 이름이 무엇이었는지 기억하시나요?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='네, 길동님이라고 하셨죠. 맞나요?', additional_kwargs={}, response_metadata={})]}

In [58]:
from operator import itemgetter
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain.memory import ConversationBufferWindowMemory

In [66]:
memory = ConversationBufferWindowMemory(k = 2, return_messages=True, memory_key='chat_history') 
# 최근 대화만 일정 개수(k)로 저장함 -> 조금 더 최근 대화에 집중 함

In [71]:
def conversation_chain(memory, query):
    chat_history = RunnablePassthrough.assign(
        chat_history = RunnableLambda(memory.load_memory_variables) | itemgetter(memory.memory_key)
    )
    llm = ChatOpenAI(model = 'gpt-4o', temperature=0)
    prompt = ChatPromptTemplate.from_messages(
        [
            ('system', '너는 친절한 AI 비서야'),
            MessagesPlaceholder(variable_name='chat_history'),
            ('human', "{input}")
        ]
    )
    chain = chat_history | prompt | llm | StrOutputParser()
    # StrOutputParser() : 출력되는 값을 자동으로 파싱(불필요한 개행문자, 특수기호 등을 자동으로 처리)
    answer = chain.invoke({'input' : query})
    memory.save_context(inputs = {'human' : query}, outputs={'ai' : answer})
    return answer

In [72]:
response = conversation_chain(
    memory = memory,
    query = '갑자기 날씨가 너무 추워졌어. 이때 먹을만한 음식을 추천해줘'
)
print('ai : ', response)

ai :  추운 날씨에는 몸을 따뜻하게 해줄 수 있는 음식을 먹는 것이 좋습니다. 다음은 추운 날씨에 어울리는 음식 몇 가지입니다:

1. **김치찌개**: 매콤하고 뜨거운 국물로 몸을 따뜻하게 해줍니다.
2. **된장찌개**: 구수한 맛과 함께 다양한 채소와 두부를 넣어 영양가도 높습니다.
3. **삼계탕**: 닭고기와 인삼, 대추 등을 넣어 끓인 국물로 보양식으로 좋습니다.
4. **칼국수**: 따뜻한 국물과 쫄깃한 면발이 잘 어울리는 음식입니다.
5. **수제비**: 밀가루 반죽을 뜯어 넣어 만든 국물 요리로, 감자나 호박 등을 넣어도 맛있습니다.
6. **어묵탕**: 어묵과 다양한 채소를 넣어 끓인 국물로 간단하면서도 든든합니다.
7. **호떡**: 달콤한 속이 들어간 따뜻한 간식으로, 추운 날씨에 간식으로 좋습니다.

이 외에도 따뜻한 차나 커피와 함께 즐길 수 있는 간단한 디저트도 좋습니다. 따뜻한 음식으로 몸을 녹이며 건강하게 겨울을 보내세요!


In [73]:
response = conversation_chain(
    memory = memory,
    query = '갑자기 날씨가 너무 추워졌어. 이때 들을만한 음악 추천해줘'
)
print('ai : ', response)

ai :  추운 날씨에는 따뜻한 분위기를 만들어 줄 수 있는 음악을 듣는 것이 좋습니다. 다음은 추운 날씨에 어울리는 음악 몇 가지 추천입니다:

1. **Norah Jones - "Come Away With Me"**: 부드럽고 감미로운 목소리가 마음을 편안하게 해줍니다.
2. **Adele - "Someone Like You"**: 감성적인 발라드로, 차분한 시간을 보내기에 좋습니다.
3. **Frank Sinatra - "The Way You Look Tonight"**: 클래식한 재즈 곡으로, 따뜻한 느낌을 줍니다.
4. **Ed Sheeran - "Perfect"**: 로맨틱한 분위기의 곡으로, 추운 날씨에 잘 어울립니다.
5. **Sara Bareilles - "Winter Song"**: 겨울의 분위기를 잘 담아낸 곡입니다.
6. **Vivaldi - "Winter" from The Four Seasons**: 클래식 음악으로, 겨울의 차가운 아름다움을 느낄 수 있습니다.
7. **Bing Crosby - "White Christmas"**: 겨울 하면 떠오르는 클래식한 크리스마스 곡입니다.

이 음악들을 들으며 따뜻한 음료와 함께 편안한 시간을 보내보세요. 음악이 주는 따뜻함이 추운 날씨를 잊게 해줄 것입니다.


In [74]:
response = conversation_chain(
    memory = memory,
    query = '갑자기 날씨가 너무 추워졌어. 이때 들을만한 한국음악 추천해줘'
)
print('ai : ', response)

ai :  추운 날씨에 어울리는 따뜻한 분위기의 한국 음악을 추천해드릴게요:

1. **김광석 - "이등병의 편지"**: 감성적인 멜로디와 가사가 마음을 따뜻하게 해줍니다.
2. **아이유 - "밤편지"**: 부드러운 목소리와 감미로운 멜로디가 잘 어우러진 곡입니다.
3. **폴킴 - "모든 날, 모든 순간"**: 사랑스러운 가사와 멜로디로 마음을 녹여줍니다.
4. **성시경 - "너의 모든 순간"**: 감미로운 발라드로, 차분한 시간을 보내기에 좋습니다.
5. **어반자카파 - "그때의 나, 그때의 우리"**: 따뜻한 감성을 느낄 수 있는 곡입니다.
6. **백예린 - "Square (2017)"**: 잔잔한 멜로디와 독특한 목소리가 매력적인 곡입니다.
7. **이문세 - "옛사랑"**: 클래식한 발라드로, 추억을 떠올리게 하는 곡입니다.

이 곡들을 들으며 따뜻한 차 한 잔과 함께 포근한 시간을 보내보세요. 음악이 주는 따뜻함이 추운 날씨를 잊게 해줄 것입니다.


In [75]:
response = conversation_chain(
    memory = memory,
    query = '아까 추천해준 음식이 뭐였지?'
)
print('ai : ', response)

ai :  죄송하지만, 이전에 음식에 대한 추천을 드린 적은 없는 것 같습니다. 추운 날씨에 어울리는 음식을 추천해드릴까요? 따뜻한 국물 요리나 뜨거운 음료가 좋을 것 같은데요. 예를 들어, 김치찌개, 된장찌개, 떡국 같은 한국 전통 음식이나 따뜻한 차나 핫초코 같은 음료를 추천드립니다. 도움이 필요하시면 언제든지 말씀해 주세요!


In [76]:
response = conversation_chain(
    memory = memory,
    query = '아까 추천해준 음악이 뭐였지?'
)
print('ai : ', response)

ai :  아까 추천해드린 음악은 추운 날씨에 어울리는 따뜻한 분위기의 한국 음악들이었습니다. 다시 말씀드리면:

1. 김광석 - "이등병의 편지"
2. 아이유 - "밤편지"
3. 폴킴 - "모든 날, 모든 순간"
4. 성시경 - "너의 모든 순간"
5. 어반자카파 - "그때의 나, 그때의 우리"
6. 백예린 - "Square (2017)"
7. 이문세 - "옛사랑"

이 곡들이 추운 날씨에 따뜻한 감성을 느끼게 해줄 수 있을 거예요.


In [77]:
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationSummaryBufferMemory

In [79]:
llm = ChatOpenAI(model = 'gpt-4o-mini')

In [80]:
memory = ConversationSummaryBufferMemory(
    llm = llm,
    max_token_limit=200,
    return_messages = True
)
# 대화내용을 요약해서 저장

C:\Users\RMARKET\AppData\Local\Temp\ipykernel_17708\2976805407.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryBufferMemory(


In [81]:
memory.save_context(
    inputs={"human": "이 식당에서 가장 인기 있는 메뉴는 무엇인가요?"},
    outputs={
        "ai": "이 식당에서 가장 인기 있는 메뉴는 불고기 정식, 해물파전, 비빔밥, 그리고 감자탕입니다. 특히 불고기는 달콤하고 짭짤한 맛으로 외국인 손님들에게도 큰 인기를 끌고 있습니다."
    },
)
memory.save_context(
    inputs={"human": "채식주의자를 위한 메뉴가 제공되나요?"},
    outputs={
        "ai": "네, 채식주의자를 위한 메뉴로 채소 비빔밥, 두부구이, 야채전, 그리고 나물 반찬 세트를 제공합니다. 신선한 제철 채소로 만들어져 건강하고 맛있는 식사를 즐기실 수 있습니다."
    },
)
memory.save_context(
    inputs={"human": "어린이를 위한 메뉴도 있나요?"},
    outputs={
        "ai": "네, 어린이를 위한 메뉴로 미니 김밥, 떡볶이, 그리고 달콤한 간장 치킨을 준비하고 있습니다. 아이들이 좋아할 만한 맛과 건강을 고려한 요리들입니다."
    },
)
memory.save_context(
    inputs={"human": "이 식당은 어떤 분위기를 가지고 있나요?"},
    outputs={
        "ai": "이 식당은 한옥 스타일의 인테리어로 전통적인 한국의 분위기를 느낄 수 있습니다. 따뜻하고 편안한 조명과 현대적인 디자인이 조화를 이루어 가족 단위 손님뿐만 아니라 연인들의 데이트 장소로도 인기가 많습니다."
    },
)

In [83]:
memory.load_memory_variables({})['history']

[SystemMessage(content='The human asks about the most popular menu items at the restaurant. The AI responds that the most popular items are bulgogi set, seafood pancake, bibimbap, and potato soup, noting that bulgogi is particularly popular among foreign guests due to its sweet and savory flavor. The human then inquires if there are menu options for vegetarians, to which the AI replies that vegetarian dishes include vegetable bibimbap, grilled tofu, vegetable pancakes, and a set of seasonal vegetable side dishes, all made with fresh, seasonal ingredients for a healthy and delicious meal.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='어린이를 위한 메뉴도 있나요?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='네, 어린이를 위한 메뉴로 미니 김밥, 떡볶이, 그리고 달콤한 간장 치킨을 준비하고 있습니다. 아이들이 좋아할 만한 맛과 건강을 고려한 요리들입니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='이 식당은 어떤 분위기를 가지고 있나요?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='이 식당은 한

In [ ]:
memory.load_memory_variables({})['history'][0].content # 대화내용을 요약본으로 저장 (30개 이상은 요약본으로 저장)

'The human asks about the most popular menu items at the restaurant. The AI responds that the most popular items are bulgogi set, seafood pancake, bibimbap, and potato soup, noting that bulgogi is particularly popular among foreign guests due to its sweet and savory flavor. The human then inquires if there are menu options for vegetarians, to which the AI replies that vegetarian dishes include vegetable bibimbap, grilled tofu, vegetable pancakes, and a set of seasonal vegetable side dishes, all made with fresh, seasonal ingredients for a healthy and delicious meal.'